In [31]:
import torch
import numpy as np

from torch.optim import Adam
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.data.dataloader import DataLoader

from preprocessor import AgeRecognitionPreprocessor
from dataset import AgeRecognitionDataset
from models import vit_l_16_age_recognizer
from loss import AgeRecognitionLoss

In [21]:
lr = 1e-3

IMAGE_DIR = './Cleaned/'
TRAINING_PAIRINGS = './training_data.csv'
BATCH_SIZE = 128

In [23]:
model = vit_l_16_age_recognizer()
loss = AgeRecognitionLoss()
preprocessor = AgeRecognitionPreprocessor()

In [18]:
optimizer = Adam(list(model.parameters()) + list(loss.parameters()), lr=lr)
scheduler = CosineAnnealingLR(optimizer=optimizer, T_max=1000, eta_min=1e-6)

In [24]:
dataset = AgeRecognitionDataset(triplet_csv_path=TRAINING_PAIRINGS, image_dir=IMAGE_DIR, preprocessor=preprocessor)
dataloader = DataLoader(dataset=dataset, batch_size=)


In [35]:
l1 = np.arange(1, 10)
l2 = np.arange(2, 4)
np.setdiff1d(l1, l2)

array([1, 2, 3, 4, 5, 6, 7, 8, 9])

In [49]:
def kfold_cross_validation(dataset, curr_fold, folds=5):
    assert curr_fold < folds, "Current fold exceeds total number of folds"
    per_partition_length = len(dataset) // folds
    all_indices = np.arange(0, len(dataset))
    validating_indices = np.arange(curr_fold * per_partition_length, (curr_fold + 1) * per_partition_length)
    training_indices = np.setdiff1d(all_indices, validating_indices)
    return training_indices, validating_indices